In [1]:
import numpy as np
import gp_grief
from scipy.stats import multivariate_normal as mvn #.logpdf
from scipy.stats import chi2
import pymcmc as pm
from numpy.testing import *
from pdb import set_trace
gp_grief.debug()

Initialize the dataset

In [2]:
np.random.seed(0)
X = np.random.randn(100, 4)
X[:, 0] = 1. # Intercept column.
Y = np.dot(X, [0.5, 0.1, 0.25, 1.]) + 0.1 * np.random.randn(X.shape[0])

Check the log-likelihood value

In [3]:
np.random.seed(0)
m = gp_grief.models.GPweb(Phi=X, y=Y)
tmp = np.random.rand(*m.parameters.shape) + 1e-6
m.parameters = tmp
ll_gpweb = m.log_likelihood()

# now compute manually
w = m.kern.parameters.reshape((1,-1))
sig2 = m.noise_var
Phi = X
K = Phi.dot(np.diag(w.squeeze()).dot(Phi.T)) + sig2*np.identity(Phi.shape[0])
ll_exact = mvn.logpdf(Y.squeeze(), mean=np.zeros(Phi.shape[0]), cov=K)

assert_array_almost_equal(ll_gpweb, ll_exact)
print "passed"

[ 14:22:49 ] gp_grief.models DEBUG: Initializing GPweb model.
passed


check the log likelihood-gradients

In [4]:
m.checkgrad()

[ 14:22:49 ] gp_grief.models INFO: Gradient check passed.


True

now wrap in pymcmc

In [5]:
mcm = gp_grief.models.PyMCMC_Wrapper(m)
print mcm
print "Printing the transformed parameters:"
print mcm.params 

# perform the check
mcm.checkgrad()


GPweb Model
| Name      |    Value | Constraint   |
|-----------+----------+--------------|
| noise_var | 0.548815 | +ve          |
Printing the transformed parameters:
[-0.31306898  0.04361093 -0.18975376 -0.32239961 -0.63953869]
[ 14:22:49 ] gp_grief.models INFO: Gradient check passed.
